In [ ]:
import os,sys,re
import csv
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from tqdm.notebook import tqdm
import json
from gensim.models import Word2Vec


In [ ]:
#base-path
path = r"/workspace"
data_path = os.path.join(path,"data")

## データのロード

In [ ]:
marge_json_path = os.path.join(path,"data/output/marge_json/")

with open(os.path.join(marge_json_path,"marge_cv.json"),"r") as f:
    cv_dict = json.load(f)

with open(os.path.join(marge_json_path,"marge_JDs.json"),"r") as f:
    JDs_dict = json.load(f)

with open(os.path.join(marge_json_path,"marge_DataEngineer.json"),"r") as f:
    DE_dict = json.load(f)

## word2vecモデルの作成

In [ ]:
#モデルに食わせるリストの作成
w2v_list = []
for key in cv_dict.keys():
    w2v_list.append(cv_dict[key]["re_token"])
for key in JDs_dict.keys():
    w2v_list.append(JDs_dict[key]["re_token"])
for key in DE_dict.keys():
    w2v_list.append(DE_dict[key]["re_token"])

In [ ]:

#モデルの作成
model = Word2Vec(sentences=w2v_list, vector_size=300, window=5, min_count=1, workers=4)
#格納用のディレクトリ作成
!mkdir -p /workspace/data/output/model
# 保存
model.save(os.path.join(path,f"data/output/model/w2v.model"))

In [ ]:
#モデルのロード
model = Word2Vec.load(os.path.join(path,f"data/output/model/w2v.model"))

## word mover's distanceの算出

### skillNerで抽出したスキルのうち、re_tokenに含まれていないものは除外

In [ ]:
#skillNerで抽出したスキルのうち、re_tokenに含まれていないものは除外
def remove_Nan_skill(_dict):
    # list = []
    cp_dict = _dict.copy()
    for key in tuple(cp_dict.keys()):
        for skill in cp_dict[key]['unique_skill']:
            if(skill not in  cp_dict[key]["re_token"]):
                cp_dict[key]["unique_skill"].remove(skill)
                # print(skill)
            #     list.append(skill)
            # cp_dict[key]['unique_skill'] = list
        if(len(cp_dict[key]["unique_skill"]) < 5): #抽出されたスキルが4つ以下のものは除外
            del cp_dict[key]
    return cp_dict

In [ ]:
r_DE_dict = remove_Nan_skill(DE_dict)
r_cv_dict = remove_Nan_skill(cv_dict)
r_JDs_dict = remove_Nan_skill(JDs_dict)

### Word Mover's Distanceの算出

In [ ]:
#cvとJDs
JDs_wmd = dict.fromkeys(cv_dict)
for cv_keys in tqdm(cv_dict.keys()):
    each_wmd_dict = {}
    for JDs_keys in [*JDs_dict.keys()]:
        each_wmd_dict[JDs_keys] = model.wv.wmdistance(cv_dict[cv_keys]['re_token'],JDs_dict[JDs_keys]['re_token'])
    JDs_wmd[cv_keys] = each_wmd_dict

In [ ]:
#cvとJDs　64分かかります
DE_wmd = dict.fromkeys(cv_dict)
for cv_keys in tqdm(cv_dict.keys()):
    each_wmd_dict = {}
    for DE_keys in DE_dict.keys():
        each_wmd_dict[DE_keys] = model.wv.wmdistance(cv_dict[cv_keys]['re_token'],DE_dict[DE_keys]['re_token'])
    DE_wmd[cv_keys] = each_wmd_dict

In [ ]:
#格納用のディレクトリの作成
!mkdir -p /workspace/data/output/ranking/WMD

In [ ]:
#dump

with open(os.path.join(data_path,'output/ranking/WMD/cv_JDs_wmd.json'),"w") as f:
    json.dump(JDs_wmd,f)
with open(os.path.join(data_path,'output/ranking/WMD/cv_DE_wmd.json'),"w") as f:
    json.dump(DE_wmd,f)